In [238]:
import tensorflow as tf
from gene_expression import *
from pathway_hierarchy import *
from utils import *
import torch

In [239]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [240]:
import pandas as pd
import numpy as np

In [241]:
tf.test.is_gpu_available()

2024-08-02 11:50:52.754423: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-02 11:50:52.754588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-02 11:50:52.754674: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

True

In [242]:
torch.cuda.is_available()


True

In [243]:
rdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_count_matrix.csv', index_col=0)
qdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_count_matrix.csv', index_col=0)
rlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_label.csv')
qlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_label.csv')

In [244]:
single_cell_metadata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/metadata_inhibitory_neurons.csv')

In [245]:
single_cell_metadata

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,individualID,clinical_diagnosis,pathological_diagnosis,clinical_pathological_AD,train_test_clinical_and_pathological
0,AAACCCAAGAAATCCA.12.9,ROSMAP,13490,4276,0.237213,0.289103,0.879183,Inh L3-5 SST MAFB,ROSMAP-65967,yes,R3857147,AD,AD,AD_with_Plaques,train
1,AAACCCAAGAACGCGT.6.6,ROSMAP,2651,1533,8.185590,0.452659,0.930517,Inh L3-5 SST MAFB,ROSMAP-90639,yes,R7090624,NCI,AD,False,NaN
2,AAACCCAAGAACTGAT.10.12,ROSMAP,6550,2764,4.809160,0.274809,0.901814,Inh LAMP5 RELN,ROSMAP-57958,yes,R2347173,NCI,AD,False,NaN
3,AAACCCAAGAAGCGGG.31.8,ROSMAP,11658,4377,5.738549,0.394579,0.895381,Inh VIP CLSTN2,ROSMAP-40761,yes,R1287407,False,AD,False,NaN
4,AAACCCAAGAATCCCT.14.8,ROSMAP,15297,4688,0.921749,0.307250,0.877260,Inh VIP CLSTN2,ROSMAP-68841,yes,R3757880,NCI,AD,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329694,TTTGTTGTCTTAGCAG.7.9,ROSMAP,9309,3459,2.062520,0.225588,0.891670,Inh VIP TSHZ2,ROSMAP-77886,no,R2554598,NCI,False,False,NaN
329695,TTTGTTGTCTTCCCGA.28.6,ROSMAP,22844,5507,0.153213,0.358956,0.858250,Inh PVALB SULF1,ROSMAP-38931,no,R6292415,NCI,No AD,NCI_with_No_Plaques,train
329696,TTTGTTGTCTTCGACC.3.13,ROSMAP,16770,5431,2.098986,0.679785,0.884093,Inh CUX2 MSR1,ROSMAP-53472,yes,R3863249,False,False,False,NaN
329697,TTTGTTGTCTTGGTGA.8.11,ROSMAP,9475,3447,0.147757,0.179420,0.889569,Inh VIP ABI3BP,ROSMAP-44788,no,R5221394,NCI,No AD,NCI_with_No_Plaques,test


In [246]:
# Verifying Train Label is correct

list(single_cell_metadata.set_index('cell_id').loc[rdata.columns,].clinical_pathological_AD) == list(rlabel.diagnosis)

True

In [247]:
# Verifying Test Label is correct

list(single_cell_metadata.set_index('cell_id').loc[qdata.columns,].clinical_pathological_AD) == list(qlabel.diagnosis)

True

In [248]:
rdata = rdata.T
qdata = qdata.T
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847
AAACCCAAGAAATCCA.12.9,0,0,0,0,0,0,0,2,0,1,...,0,3,2,1,0,0,3,0,4,0
AAACCCAAGATTCGCT.20.4,0,0,0,0,0,0,0,1,0,1,...,0,40,34,6,0,12,4,0,14,0
AAACCCAAGCCATCCG.21.6,0,0,0,0,0,0,0,0,0,0,...,0,4,5,1,0,3,0,0,1,0
AAACCCAAGCCTATTG.18.6,0,0,0,0,0,0,0,0,0,0,...,0,23,40,11,2,21,2,0,18,0
AAACCCAAGCTAGATA.25.10,0,1,0,0,0,1,0,0,0,0,...,1,90,118,43,0,32,10,0,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCACACCGG.11.9,0,0,0,0,0,0,0,0,0,0,...,1,27,47,11,1,8,6,0,19,0
TTTGTTGTCCGAAATC.9.14,0,0,0,0,0,0,0,0,0,1,...,0,4,4,0,0,4,0,0,3,0
TTTGTTGTCGAACCTA.5.2,0,0,0,0,0,0,0,0,0,0,...,0,15,33,3,0,6,2,0,5,0
TTTGTTGTCTTACACT.27.3,0,0,0,0,0,0,0,0,0,1,...,1,21,36,9,1,7,4,0,25,0


In [249]:
rdata['diagnosis'] =  list(rlabel.diagnosis)
qdata['diagnosis'] =  list(qlabel.diagnosis)

In [250]:
rdata.diagnosis.value_counts()

diagnosis
NCI_with_No_Plaques    29147
AD_with_Plaques        24372
Name: count, dtype: int64

In [251]:
rdata_neg = rdata[rdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:24372,]
rdata_pos = rdata[rdata.diagnosis == 'AD_with_Plaques'].iloc[0:24372,]
rdata = pd.concat([rdata_neg,rdata_pos])


In [252]:
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
AAACCCAAGATTCGCT.20.4,0,0,0,0,0,0,0,1,0,1,...,40,34,6,0,12,4,0,14,0,NCI_with_No_Plaques
AAACCCAAGCTAGTTC.31.9,0,0,0,0,0,0,0,0,0,2,...,40,67,10,0,35,2,0,25,0,NCI_with_No_Plaques
AAACCCAAGGTGCTAG.19.10,0,0,0,0,0,0,0,0,0,0,...,7,8,4,0,1,0,0,3,0,NCI_with_No_Plaques
AAACCCACAACAAGAT.11.11,0,0,0,0,0,0,0,0,0,0,...,13,13,4,0,8,0,0,3,0,NCI_with_No_Plaques
AAACCCACAAGTGGTG.5.2,0,0,0,0,0,1,0,0,0,1,...,29,40,5,0,25,5,0,15,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGACACAG.4.13,0,0,0,0,1,1,0,1,0,0,...,6,26,6,1,6,0,0,8,0,AD_with_Plaques
TTTGTTGGTGAGCTCC.13.12,0,0,0,0,0,0,0,0,0,0,...,19,26,3,0,18,1,0,7,0,AD_with_Plaques
TTTGTTGGTGGATCGA.32.3,0,1,0,0,0,0,0,0,0,0,...,14,4,17,1,3,2,0,1,0,AD_with_Plaques
TTTGTTGTCACACCGG.11.9,0,0,0,0,0,0,0,0,0,0,...,27,47,11,1,8,6,0,19,0,AD_with_Plaques


In [253]:
rdata = rdata.sample(frac = 1)

In [254]:
rdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
CTTCCGATCAGACATC.8.14,0,0,0,1,0,0,2,0,0,2,...,149,171,35,1,64,13,0,66,0,AD_with_Plaques
CGTAGTAAGGGCTGAT.32.2,0,0,0,0,1,0,0,0,0,1,...,6,11,0,0,1,0,0,3,0,NCI_with_No_Plaques
CTCCTCCTCCTTCAGC.11.11,0,0,0,0,0,0,0,1,0,0,...,11,15,3,1,6,4,0,8,0,NCI_with_No_Plaques
CGAGAAGAGACTCAAA.12.10,0,0,0,0,1,1,0,1,0,0,...,159,206,87,2,96,8,0,60,0,NCI_with_No_Plaques
CTCACACTCCTGTAGA.32.1,0,0,0,0,0,0,0,0,0,1,...,12,22,12,1,16,2,0,13,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TAAACCGCAGCCAGAA.5.1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,AD_with_Plaques
GAAACTCTCCTTAATC.17.1,0,0,0,0,0,0,0,0,0,0,...,3,6,1,0,6,1,0,3,0,NCI_with_No_Plaques
CAGAGCCTCCGAGAAG.27.11,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,1,0,NCI_with_No_Plaques
AATTCCTGTTTGACAC.16.4,0,0,0,0,0,1,0,1,0,1,...,67,87,9,1,37,5,0,24,0,AD_with_Plaques


In [255]:
rlabel = pd.DataFrame(rdata.diagnosis)

In [256]:
rlabel

,diagnosis
CTTCCGATCAGACATC.8.14,AD_with_Plaques
CGTAGTAAGGGCTGAT.32.2,NCI_with_No_Plaques
CTCCTCCTCCTTCAGC.11.11,NCI_with_No_Plaques
CGAGAAGAGACTCAAA.12.10,NCI_with_No_Plaques
CTCACACTCCTGTAGA.32.1,NCI_with_No_Plaques
...,...
TAAACCGCAGCCAGAA.5.1,AD_with_Plaques
GAAACTCTCCTTAATC.17.1,NCI_with_No_Plaques
CAGAGCCTCCGAGAAG.27.11,NCI_with_No_Plaques
AATTCCTGTTTGACAC.16.4,AD_with_Plaques


In [257]:
rdata = rdata.drop(columns=['diagnosis'])

In [258]:
qdata.diagnosis.value_counts()

diagnosis
NCI_with_No_Plaques    5241
AD_with_Plaques        3442
Name: count, dtype: int64

In [259]:
qdata_neg = qdata[qdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:3442,]
qdata_pos =qdata[qdata.diagnosis == 'AD_with_Plaques'].iloc[0:3442,]
qdata = pd.concat([qdata_neg,qdata_pos])

In [260]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
AAACCCAAGGGCCCTT.21.11,0,0,0,0,0,0,0,0,0,0,...,4,12,4,0,2,4,0,4,0,NCI_with_No_Plaques
AAACCCACAACGGCCT.32.11,0,1,0,0,0,1,2,0,0,2,...,5,5,0,0,0,0,0,3,0,NCI_with_No_Plaques
AAACCCACACAAGCAG.42.2,0,0,0,0,0,1,0,0,0,0,...,58,79,12,5,32,6,0,38,0,NCI_with_No_Plaques
AAACCCAGTGTACGCC.21.11,0,0,0,1,0,0,0,0,0,0,...,18,24,5,0,11,1,0,10,0,NCI_with_No_Plaques
AAACCCATCAGACCCG.6.2,0,0,0,0,0,0,1,0,0,0,...,24,29,13,3,17,5,0,8,0,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTGTCGCATCG.38.1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,2,0,AD_with_Plaques
TTTGGTTGTGCCTGGT.38.1,0,0,0,0,0,0,0,0,0,0,...,2,5,1,0,2,0,0,0,0,AD_with_Plaques
TTTGGTTTCTGCAGCG.34.2,0,0,0,0,0,0,0,0,0,2,...,3,1,2,0,2,1,0,4,0,AD_with_Plaques
TTTGTCAGTCAGGACA.38.1,0,0,0,0,0,0,0,0,0,0,...,4,23,6,0,9,1,0,8,0,AD_with_Plaques


In [261]:
qdata = qdata.sample(frac = 1)

In [262]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847,diagnosis
GAAGCCCTCGGTATGT.13.11,0,0,0,0,0,0,0,0,0,1,...,12,29,5,0,22,1,0,20,0,NCI_with_No_Plaques
CGCCAGACAGACCATT.21.11,0,0,0,0,0,0,0,0,0,0,...,4,10,6,0,2,2,1,8,0,NCI_with_No_Plaques
CTACATTTCCTAGCGG.34.2,0,0,0,0,0,0,0,0,0,0,...,67,60,26,3,19,3,0,21,0,AD_with_Plaques
AGTGATCAGGGTATAT.6.2,0,0,0,0,0,0,0,0,0,0,...,3,10,0,0,8,0,0,3,0,NCI_with_No_Plaques
AGCATACTCACAAACC.38.1,0,0,0,0,0,0,0,0,0,0,...,9,21,3,0,11,1,0,7,0,AD_with_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GAAGGGTGTCCTGAAT.32.11,0,1,0,0,0,0,0,0,0,0,...,7,5,0,0,6,1,0,3,0,NCI_with_No_Plaques
ATACCGAAGTTGCGCC.21.11,0,0,0,0,0,0,0,0,0,0,...,7,6,2,0,4,1,0,2,0,NCI_with_No_Plaques
AACCTTTTCTTTGCTA.13.11,0,0,0,0,0,0,0,0,0,1,...,6,7,1,0,2,0,0,0,0,NCI_with_No_Plaques
AGACAGGGTATCACGT.21.11,0,0,0,0,0,0,1,1,0,0,...,24,23,13,1,5,3,0,11,0,NCI_with_No_Plaques


In [263]:
qlabel = pd.DataFrame(qdata.diagnosis)

In [264]:
qlabel.reset_index(drop = True, inplace = True)
rlabel.reset_index(drop = True, inplace = True)
rlabel

,diagnosis
0,AD_with_Plaques
1,NCI_with_No_Plaques
2,NCI_with_No_Plaques
3,NCI_with_No_Plaques
4,NCI_with_No_Plaques
...,...
48739,AD_with_Plaques
48740,NCI_with_No_Plaques
48741,NCI_with_No_Plaques
48742,AD_with_Plaques


In [265]:
qdata = qdata.drop(columns=['diagnosis'])

In [266]:
qdata

ensemble_gene_name,ENSG00000163463,ENSG00000163462,ENSG00000185499,ENSG00000169231,ENSG00000261905,ENSG00000263290,ENSG00000261893,ENSG00000263324,ENSG00000262785,ENSG00000160752,...,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000274847
GAAGCCCTCGGTATGT.13.11,0,0,0,0,0,0,0,0,0,1,...,0,12,29,5,0,22,1,0,20,0
CGCCAGACAGACCATT.21.11,0,0,0,0,0,0,0,0,0,0,...,0,4,10,6,0,2,2,1,8,0
CTACATTTCCTAGCGG.34.2,0,0,0,0,0,0,0,0,0,0,...,2,67,60,26,3,19,3,0,21,0
AGTGATCAGGGTATAT.6.2,0,0,0,0,0,0,0,0,0,0,...,0,3,10,0,0,8,0,0,3,0
AGCATACTCACAAACC.38.1,0,0,0,0,0,0,0,0,0,0,...,0,9,21,3,0,11,1,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GAAGGGTGTCCTGAAT.32.11,0,1,0,0,0,0,0,0,0,0,...,0,7,5,0,0,6,1,0,3,0
ATACCGAAGTTGCGCC.21.11,0,0,0,0,0,0,0,0,0,0,...,1,7,6,2,0,4,1,0,2,0
AACCTTTTCTTTGCTA.13.11,0,0,0,0,0,0,0,0,0,1,...,0,6,7,1,0,2,0,0,0,0
AGACAGGGTATCACGT.21.11,0,0,0,0,0,0,1,1,0,0,...,0,24,23,13,1,5,3,0,11,0


In [267]:
rdata = rdata.T
qdata = qdata.T

In [268]:
r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()

In [269]:
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  print(len(mkh))
  print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  print(mkh)
  print(mkl)
  return mkh, mkl


def get_expression(rdata, qdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    qdata = qdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
    return train_x, test_x, train_y


In [270]:

r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.90,
                                              thrl=0.90,
                                              normalization=True,
                                              marker=True)

2250
2250
4500
4500
      AD_with_Plaques NCI_with_No_Plaques
0     ENSG00000160752     ENSG00000132718
1     ENSG00000163479     ENSG00000160789
2     ENSG00000163468     ENSG00000162734
3     ENSG00000143256     ENSG00000248485
4     ENSG00000143252     ENSG00000198929
...               ...                 ...
2245  ENSG00000198804     ENSG00000198712
2246  ENSG00000228253     ENSG00000198899
2247  ENSG00000198938     ENSG00000198840
2248  ENSG00000212907     ENSG00000198886
2249  ENSG00000198786     ENSG00000198727

[2250 rows x 2 columns]
      AD_with_Plaques NCI_with_No_Plaques
0     ENSG00000160752     ENSG00000132718
1     ENSG00000163479     ENSG00000160789
2     ENSG00000163468     ENSG00000162734
3     ENSG00000143256     ENSG00000248485
4     ENSG00000143252     ENSG00000198929
...               ...                 ...
2245  ENSG00000198804     ENSG00000198712
2246  ENSG00000228253     ENSG00000198899
2247  ENSG00000198938     ENSG00000198840
2248  ENSG00000212907     ENSG0

In [271]:
train_x

,CTTCCGATCAGACATC.8.14,CGTAGTAAGGGCTGAT.32.2,CTCCTCCTCCTTCAGC.11.11,CGAGAAGAGACTCAAA.12.10,CTCACACTCCTGTAGA.32.1,AACGGGAGTTCGTGCG.18.6,ATTCTACCACTTGACA.1.6,TCGATTTAGACGAAGA.26.2,ATTTCACGTACCGTGC.5.11,CTTTGCGTCGGCCGAT.22.1,...,CTTCTAATCAATCAGC.14.4,TGTGATGTCTGATGGT.11.9,GTTGCGGAGTTAGTAG.11.11,TCAATCTGTGTACGCC.27.11,TCGAGGCTCAGCTCTC.19.0,TAAACCGCAGCCAGAA.5.1,GAAACTCTCCTTAATC.17.1,CAGAGCCTCCGAGAAG.27.11,AATTCCTGTTTGACAC.16.4,CGTGATACAGAGTTCT.30.6
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000001461,0.000000,0.000000,0.000000,0.420977,0.000000,1.997191,0.000000,0.000000,1.953969,0.000000,...,0.000000,2.047559,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.667503,0.000000
ENSG00000002746,1.847876,1.878677,1.018903,2.133674,2.262707,3.084158,0.000000,0.000000,1.953969,0.000000,...,2.050793,2.297947,0.000000,0.000000,0.000000,2.173917,2.468086,1.941448,2.783083,1.502936
ENSG00000003056,0.831804,0.000000,0.000000,0.746457,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.912637,0.000000
ENSG00000003147,0.831804,2.667836,1.610112,1.429840,1.820733,0.000000,1.336201,1.887583,1.953969,0.000000,...,1.034060,0.000000,2.090069,0.000000,0.000000,0.000000,0.000000,1.941448,1.866888,2.222922
ENSG00000004059,0.333389,0.000000,0.000000,0.420977,1.180341,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.271351,0.000000,0.000000,0.000000,0.372017,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000292158,0.831804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000292253,0.831804,0.000000,0.000000,1.235932,0.000000,0.000000,0.000000,0.000000,1.953969,0.000000,...,0.000000,0.000000,1.060666,0.000000,2.753097,0.000000,0.000000,0.000000,0.372017,0.000000
ENSG00000292308,0.000000,0.000000,0.000000,0.000000,0.000000,1.319682,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [272]:
train_y

,diagnosis
CTTCCGATCAGACATC.8.14,AD_with_Plaques
CGTAGTAAGGGCTGAT.32.2,NCI_with_No_Plaques
CTCCTCCTCCTTCAGC.11.11,NCI_with_No_Plaques
CGAGAAGAGACTCAAA.12.10,NCI_with_No_Plaques
CTCACACTCCTGTAGA.32.1,NCI_with_No_Plaques
...,...
TAAACCGCAGCCAGAA.5.1,AD_with_Plaques
GAAACTCTCCTTAATC.17.1,NCI_with_No_Plaques
CAGAGCCTCCGAGAAG.27.11,NCI_with_No_Plaques
AATTCCTGTTTGACAC.16.4,AD_with_Plaques


In [273]:
train_y.diagnosis.value_counts()

diagnosis
AD_with_Plaques        24372
NCI_with_No_Plaques    24372
Name: count, dtype: int64

In [274]:
test_x

,GAAGCCCTCGGTATGT.13.11,CGCCAGACAGACCATT.21.11,CTACATTTCCTAGCGG.34.2,AGTGATCAGGGTATAT.6.2,AGCATACTCACAAACC.38.1,CTCAACCTCCGATCTC.26.3,CTGATCCCAATAGAGT.38.1,CTGATCCAGTTTAGGA.21.11,TGGGAGAAGTGCTCAT.34.2,AAAGTCCAGACCATAA.42.2,...,TAACCAGTCACAGTGT.10.2,CTGCGAGGTAGGATAT.6.2,GACCCTTGTATCGCTA.6.2,CTATAGGGTAGAGTTA.13.11,CGGTCAGAGACAGCTG.8.11,GAAGGGTGTCCTGAAT.32.11,ATACCGAAGTTGCGCC.21.11,AACCTTTTCTTTGCTA.13.11,AGACAGGGTATCACGT.21.11,ACGATACAGTCTCCTC.18.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000001461,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.086681,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0
ENSG00000002746,0.000000,2.873639,0.000000,0.0,0.0,0.000000,3.344475,3.484389,2.972136,3.740099,...,0.000000,2.046360,1.763623,0.0,2.201137,1.167925,0.0,0.000000,2.64604,0.0
ENSG00000003056,0.000000,0.000000,0.000000,0.0,0.0,0.633307,0.000000,0.000000,0.000000,0.643324,...,1.616412,0.834029,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0
ENSG00000003147,3.082066,2.873639,1.191579,0.0,0.0,0.633307,0.000000,0.887002,0.000000,1.699417,...,1.023651,0.834029,0.000000,0.0,1.887062,0.000000,0.0,2.586767,2.64604,0.0
ENSG00000004059,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000292158,0.000000,0.000000,0.000000,0.0,0.0,0.633307,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0
ENSG00000292253,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.023651,0.000000,1.136026,0.0,0.925699,1.167925,0.0,0.000000,0.00000,0.0
ENSG00000292308,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.023651,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0


In [275]:
ensembl_pathway_relation = '../../usman/CellTICS/reactome/Ensembl2Reactome_All_Levels.txt'
species = 'human'
pathway_names = '../../usman/CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../usman/CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'
n_hidden = 5

In [276]:

def get_gene_pathways(input_file, species='mouse'):
    dt = pd.read_table(input_file, header=None)
    print(dt.head())
    if species == 'mouse':
        ensembl = dt[(dt[0] >= 'ENSMUSG00000000000') & (dt[0] <= 'ENSMUST99999999999')]
    elif species == 'human':
        ensembl = dt[((dt[0] >= 'ENSG00000000000000') & (dt[0] <= 'ENSG00099999999999'))
                     | ((dt[0] >= 'ENSP00000000000000') & (dt[0] <= 'ENSP00099999999999'))
                     | ((dt[0] >= 'ENST00000000000000') & (dt[0] <= 'ENST00099999999999'))]
    elif species == 'rat':
        ensembl = dt[(dt[0] >= 'ENSRNOG00000000000') & (dt[0] <= 'ENSRNOT99999999999')]

    ensembl = ensembl.iloc[:, 0:2]
    ensembl.columns = ['gene', 'group']
    ensembl = pd.DataFrame(ensembl, columns=['group', 'gene'])
    ensembl.index = range(0, ensembl.shape[0])

    if species == 'mouse':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSMUSG' + ensembl.iloc[i, 1][7:18]
    elif species == 'human':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSG000' + ensembl.iloc[i, 1][7:18]
    elif species == 'rat':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSRNOG' + ensembl.iloc[i, 1][7:18]
    ensembl = ensembl.drop_duplicates()
    ensembl.index = range(0, ensembl.shape[0])
    return ensembl

In [277]:
pathway_genes = get_gene_pathways(ensembl_pathway_relation, species=species)

       0             1                                                  2  \
0  26364  R-BTA-162582  https://reactome.org/PathwayBrowser/#/R-BTA-16...   
1  26364  R-BTA-195253  https://reactome.org/PathwayBrowser/#/R-BTA-19...   
2  26364  R-BTA-195721  https://reactome.org/PathwayBrowser/#/R-BTA-19...   
3  26364  R-BTA-196299  https://reactome.org/PathwayBrowser/#/R-BTA-19...   
4  26364  R-BTA-201681  https://reactome.org/PathwayBrowser/#/R-BTA-20...   

                                                   3    4           5  
0                                Signal Transduction  IEA  Bos taurus  
1  Degradation of beta-catenin by the destruction...  IEA  Bos taurus  
2                                   Signaling by WNT  IEA  Bos taurus  
3               Beta-catenin phosphorylation cascade  IEA  Bos taurus  
4         TCF dependent signaling in response to WNT  IEA  Bos taurus  


In [278]:
pathway_genes

,group,gene
0,R-HSA-162699,ENSG00000000419
1,R-HSA-163125,ENSG00000000419
2,R-HSA-1643685,ENSG00000000419
3,R-HSA-3781865,ENSG00000000419
4,R-HSA-392499,ENSG00000000419
...,...,...
756483,R-HSA-9678110,ENSG00000680121
756484,R-HSA-9679191,ENSG00000680121
756485,R-HSA-9679506,ENSG00000680121
756486,R-HSA-9694516,ENSG00000680121


In [329]:
import numpy as np
import pandas as pd
import networkx as nx


def get_gene_pathways(input_file, species='mouse'):
    dt = pd.read_table(input_file, header=None)
    if species == 'mouse':
        ensembl = dt[(dt[0] >= 'ENSMUSG00000000000') & (dt[0] <= 'ENSMUST99999999999')]
    elif species == 'human':
        ensembl = dt[((dt[0] >= 'ENSG00000000000000') & (dt[0] <= 'ENSG00099999999999'))
                     | ((dt[0] >= 'ENSP00000000000000') & (dt[0] <= 'ENSP00099999999999'))
                     | ((dt[0] >= 'ENST00000000000000') & (dt[0] <= 'ENST00099999999999'))]
    elif species == 'rat':
        ensembl = dt[(dt[0] >= 'ENSRNOG00000000000') & (dt[0] <= 'ENSRNOT99999999999')]

    ensembl = ensembl.iloc[:, 0:2]
    ensembl.columns = ['gene', 'group']
    ensembl = pd.DataFrame(ensembl, columns=['group', 'gene'])
    ensembl.index = range(0, ensembl.shape[0])

    if species == 'mouse':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSMUSG' + ensembl.iloc[i, 1][7:18]
    elif species == 'human':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSG000' + ensembl.iloc[i, 1][7:18]
    elif species == 'rat':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSRNOG' + ensembl.iloc[i, 1][7:18]
    ensembl = ensembl.drop_duplicates()
    ensembl.index = range(0, ensembl.shape[0])
    return ensembl


class Reactome():

    def __init__(self, pathway_names, pathway_genes, relations_file_name, species):
        self.pathway_names = self.load_names(pathway_names)
        self.pathway_genes = pathway_genes
        self.hierarchy = self.load_hierarchy(relations_file_name)
        self.species = species

    def load_names(self, pathway_names):
        filename = pathway_names
        df = pd.read_csv(filename, sep='\t')
        df.columns = ['reactome_id', 'pathway_name', 'species']
        return df

    def load_hierarchy(self, relations_file_name):
        filename = relations_file_name
        df = pd.read_table(filename, header=None)
        df.columns = ['child', 'parent']
        return df


class ReactomeNetwork():

    def __init__(self, pathway_names, pathway_genes, relations_file_name, species):
        self.reactome = Reactome(pathway_names, pathway_genes, relations_file_name, species)  # low level access to reactome pathways and genes
        self.netx = self.get_reactome_networkx()

    def get_reactome_networkx(self):
        if hasattr(self, 'netx'):
            return self.netx
        hierarchy = self.reactome.hierarchy
        # filter hierarchy to have mouse pathways only
        if self.reactome.species == 'mouse':
            abbr = 'MMU'
        elif self.reactome.species == 'human':
            abbr = 'HSA'
        elif self.reactome.species == 'rat':
            abbr = 'RNO'
        print(abbr)
        species_hierarchy = hierarchy[hierarchy['child'].str.contains(abbr)]
        net = nx.from_pandas_edgelist(species_hierarchy, 'child', 'parent', create_using=nx.DiGraph())
        net.name = 'reactome'

        # add root node
        roots = [n for n, d in net.in_degree() if d == 0]
        root_node = 'root'
        edges = [(root_node, n) for n in roots]
        net.add_edges_from(edges)

        return net


def add_duplicated_edges(G, node, n_levels):
    edges = []
    source = node
    for l in range(n_levels):
        target = node + '_copy' + str(l + 1)
        edge = (source, target)
        source = target
        edges.append(edge)

    G.add_edges_from(edges)
    return G, target


def add_gene_edges(G, node, pathways, genes_df):
    genes = []
    if type(pathways) == str:
        genes = genes + list(genes_df[genes_df['group'] == pathways]['gene'])
    else:
        for i in range(len(pathways)):
            genes = genes + list(genes_df[genes_df['group'] == pathways[i]]['gene'])
    genes = list(set(genes))
    edges = []
    source = node
    for target in genes:
        edge = (source, target)
        edges.append(edge)
    G.add_edges_from(edges)
    return G


def get_nodes_at_level(net, distance):
    nodes = set(nx.ego_graph(net, 'root', radius=distance))     # get all nodes within distance around the query node
    if distance >= 1.:
        nodes -= set(nx.ego_graph(net, 'root', radius=distance - 1))     # remove nodes that are not at the specified distance but closer
    return list(nodes)


def gene_mapping(gene, df):
    inter_gene = list(set(gene) & set(df['gene']))
    genedict = {}
    genelist = [df.iloc[0, 0]]
    print(genelist)
    genedict[df.iloc[0, 1]] = genelist
    print(genedict)
    for i in range(1, df.shape[0]):
        if df.iloc[i, 1] == df.iloc[i - 1, 1]:
            genelist.append(df.iloc[i, 0])
        else:
            genedict[df.iloc[i - 1, 1]] = genelist
            genelist = [df.iloc[i, 0]]
    
    mappingdf = pd.DataFrame(data=None, columns=['group', 'gene'])
    for j in range(len(inter_gene)):
        mappingdf_iter = {'group': genedict[inter_gene[j]],
                  'gene': [inter_gene[j]] * len(genedict[inter_gene[j]])}
        mappingdf_iter = pd.DataFrame(mappingdf_iter)
        mappingdf = pd.concat([mappingdf, mappingdf_iter])
    print(mappingdf.head())
    return mappingdf

def add_duplicated_edges(G, node, n_levels):
    edges = []
    source = node
    for l in range(n_levels):
        target = node + '_copy' + str(l + 1)
        edge = (source, target)
        source = target
        edges.append(edge)

    G.add_edges_from(edges)
    return G, target

def get_nodes_at_level(net, distance):
    nodes = set(nx.ego_graph(net, 'root', radius=distance))     # get all nodes within distance around the query node
    if distance >= 1.:
        nodes -= set(nx.ego_graph(net, 'root', radius=distance - 1))     # remove nodes that are not at the specified distance but closer
    return list(nodes)


def get_masking(pathway_names, pathway_genes, relations_file_name, train_x, test_x, train_y, datatype, species='mouse', n_hidden=5):
    reactome_net = ReactomeNetwork(pathway_names, pathway_genes, relations_file_name, species)
    genes_df = reactome_net.reactome.pathway_genes
    genes_df = gene_mapping(train_x.index.tolist(), genes_df)

    original_network = reactome_net.netx
    original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]
    in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
    while in_genes_df.count(False) > 0:
        in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
        for i in range(len(original_terminal_nodes)):
            if original_terminal_nodes[i] in genes_df['group'].to_list():
                in_genes_df[i] = True
        for i in range(len(original_terminal_nodes)):
            if in_genes_df[i] == False:
                original_network.remove_node(original_terminal_nodes[i])
        original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]

    sub_graph = nx.ego_graph(original_network, 'root', radius=n_hidden) #subgraph of neighbors centered at node "root" <= a given radius (n_level).
    sub_terminal_nodes = [n for n, d in sub_graph.out_degree() if d == 0]
    
    for node in sub_terminal_nodes:
        distance = len(nx.shortest_path(sub_graph, source='root', target=node)) #len of distance: num of nodes in the shortest path
        if (distance == n_hidden + 1) & (node not in original_terminal_nodes):
            part_graph = nx.ego_graph(original_network, node, radius=100)
            corresponding_terminal_nodes = [n for n, d in part_graph.out_degree() if d == 0]
            sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
        elif (distance == n_hidden + 1) & (node in original_terminal_nodes):
            corresponding_terminal_nodes = node
            sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
        elif distance <= n_hidden:
            diff = n_hidden - distance + 1
            sub_graph, copy_node = add_duplicated_edges(sub_graph, node, diff)
            sub_graph = add_gene_edges(sub_graph, copy_node, node, genes_df)
    final_network = sub_graph

    n_level = n_hidden + 2
    layers_node = {}
    for i in range(n_level):
        nodes = get_nodes_at_level(final_network, i)
        layers_node[i] = nodes
    layers_node[0] = list(set(train_y[datatype]))

    layers_relationship = []
    for i in range(n_hidden):
        nodes = get_nodes_at_level(final_network, i + 1)
        dict = {}
        for n in nodes:
            next = list(final_network.successors(n))
            dict[n] = next
        layers_relationship.append(dict)

    masking = {}
    masking[0] = pd.DataFrame(data=1, index=layers_node[0], columns=layers_node[1])
    for i in range(1, n_level-1):
        masking[i] = pd.DataFrame(data=0, index=layers_node[i], columns=layers_node[i + 1])
        for n in layers_node[i]:
            for node in layers_relationship[i - 1][n]:
                if node in layers_node[i + 1]:
                    masking[i].loc[n, node] = 1
    print(masking[0])
    print(len(masking))
    print(masking[5])
    gene_in_network = list(masking[n_hidden].columns)
    train_x = train_x.loc[gene_in_network, :]
    test_x = test_x.loc[gene_in_network, :]
    #val_x = val_x.loc[gene_in_network, :]
    for i in range(len(masking) - 1):
        masking[i] = np.array(masking[i + 1])
    del masking[len(masking) - 1]
    return masking, layers_node, train_x, test_x
    #val_x


In [330]:
G = nx.path_graph(5)

In [331]:
G.edges

EdgeView([(0, 1), (1, 2), (2, 3), (3, 4)])

In [332]:
p = nx.shortest_path(G, source=2, target=4) 
p

[2, 3, 4]

In [333]:
len(p)

3

In [334]:
[pathway_genes.iloc[0, 0]]

['R-HSA-162699']

In [335]:
species = 'human'
pathway_names = '../../usman/CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../usman/CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'
n_hidden = 5

masking, layers_node, train_x, test_x = get_masking(pathway_names,
                                                        pathway_genes,
                                                        pathway_relation,
                                                        train_x,
                                                        test_x,
                                                        train_y,
                                                        datatype,
                                                        species,
                                                        n_hidden)

HSA
['R-HSA-162699']
{'ENSG00000000419': ['R-HSA-162699']}
          group             gene
0  R-HSA-112126  ENSG00000112249
1   R-HSA-73894  ENSG00000112249
2   R-HSA-73942  ENSG00000112249
3   R-HSA-73943  ENSG00000112249
0  R-HSA-110330  ENSG00000103152
                     R-HSA-1640170  R-HSA-1266738  R-HSA-9748784  \
AD_with_Plaques                  1              1              1   
NCI_with_No_Plaques              1              1              1   

                     R-HSA-162582  R-HSA-69306  R-HSA-5653656  R-HSA-1643685  \
AD_with_Plaques                 1            1              1              1   
NCI_with_No_Plaques             1            1              1              1   

                     R-HSA-168256  R-HSA-397014  R-HSA-5357801  ...  \
AD_with_Plaques                 1             1              1  ...   
NCI_with_No_Plaques             1             1              1  ...   

                     R-HSA-112316  R-HSA-109582  R-HSA-4839726  R-HSA-1474244  \
AD

In [282]:
reactome_net = ReactomeNetwork(pathway_names, pathway_genes, pathway_relation, species)

HSA


In [283]:
genes_df = reactome_net.reactome.pathway_genes
genes_df

,group,gene
0,R-HSA-162699,ENSG00000000419
1,R-HSA-163125,ENSG00000000419
2,R-HSA-1643685,ENSG00000000419
3,R-HSA-3781865,ENSG00000000419
4,R-HSA-392499,ENSG00000000419
...,...,...
756483,R-HSA-9678110,ENSG00000680121
756484,R-HSA-9679191,ENSG00000680121
756485,R-HSA-9679506,ENSG00000680121
756486,R-HSA-9694516,ENSG00000680121


In [284]:
hierarchy_df = reactome_net.reactome.hierarchy

In [285]:
hierarchy_df = hierarchy_df[hierarchy_df['child'].str.contains('HSA')]

In [286]:
len(set(hierarchy_df.child))

766

In [287]:
len(set(hierarchy_df.parent))

2551

In [288]:
len(set(hierarchy_df.child).intersection(set(hierarchy_df.parent)))

737

In [289]:
len(set(hierarchy_df.parent)-set(hierarchy_df.child))

1814

In [290]:
genes_df = gene_mapping(train_x.index.tolist(), genes_df)
genes_df

['R-HSA-162699']
{'ENSG00000000419': ['R-HSA-162699']}
          group             gene
0  R-HSA-112126  ENSG00000112249
1   R-HSA-73894  ENSG00000112249
2   R-HSA-73942  ENSG00000112249
3   R-HSA-73943  ENSG00000112249
0  R-HSA-110330  ENSG00000103152


,group,gene
0,R-HSA-112126,ENSG00000112249
1,R-HSA-73894,ENSG00000112249
2,R-HSA-73942,ENSG00000112249
3,R-HSA-73943,ENSG00000112249
0,R-HSA-110330,ENSG00000103152
...,...,...
0,R-HSA-1428517,ENSG00000198888
1,R-HSA-1430728,ENSG00000198888
2,R-HSA-163200,ENSG00000198888
3,R-HSA-611105,ENSG00000198888


In [291]:
len(set(genes_df.gene))

1242

In [292]:
original_network = reactome_net.netx
original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]

In [293]:
original_terminal_nodes[0:5]

['R-HSA-75205', 'R-HSA-75892', 'R-HSA-111452', 'R-HSA-111453', 'R-HSA-114294']

In [294]:
len(set(genes_df.group))

1771

In [295]:
len(original_terminal_nodes)

1814

In [296]:
len(set(genes_df.group).intersection(set(original_terminal_nodes)))

1103

In [297]:
len(set(genes_df.group)- set(original_terminal_nodes))

668

In [298]:
len(set(original_terminal_nodes) - set(genes_df.group) )

711

In [299]:
#pd.DataFrame(test_x.index).to_csv('../../preprocessed_data/inhibitory_neuron/marker_genes.csv', index=False)

In [300]:
in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
original_terminal_nodes 

['R-HSA-75205',
 'R-HSA-75892',
 'R-HSA-111452',
 'R-HSA-111453',
 'R-HSA-114294',
 'R-HSA-75108',
 'R-HSA-165160',
 'R-HSA-165158',
 'R-HSA-110312',
 'R-HSA-110320',
 'R-HSA-5655862',
 'R-HSA-5656121',
 'R-HSA-5656169',
 'R-HSA-110362',
 'R-HSA-5651801',
 'R-HSA-73930',
 'R-HSA-9627069',
 'R-HSA-111459',
 'R-HSA-264870',
 'R-HSA-351906',
 'R-HSA-352238',
 'R-HSA-111463',
 'R-HSA-111464',
 'R-HSA-111457',
 'R-HSA-180024',
 'R-HSA-202040',
 'R-HSA-163615',
 'R-HSA-111932',
 'R-HSA-111957',
 'R-HSA-111995',
 'R-HSA-170660',
 'R-HSA-170670',
 'R-HSA-112303',
 'R-HSA-181429',
 'R-HSA-181430',
 'R-HSA-210500',
 'R-HSA-212676',
 'R-HSA-264642',
 'R-HSA-210455',
 'R-HSA-112308',
 'R-HSA-112412',
 'R-HSA-110056',
 'R-HSA-112411',
 'R-HSA-113501',
 'R-HSA-113507',
 'R-HSA-111446',
 'R-HSA-111447',
 'R-HSA-111448',
 'R-HSA-139910',
 'R-HSA-139915',
 'R-HSA-426048',
 'R-HSA-1169091',
 'R-HSA-1169092',
 'R-HSA-2025928',
 'R-HSA-1169408',
 'R-HSA-8983711',
 'R-HSA-1433617',
 'R-HSA-1300642',
 'R-HS

In [301]:
# matchig data with dataset
k = 0
while in_genes_df.count(False) > 0:
    k = k+1
    in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
    for i in range(len(original_terminal_nodes)):
        if original_terminal_nodes[i] in genes_df['group'].to_list():
            in_genes_df[i] = True
    for i in range(len(original_terminal_nodes)):
        if in_genes_df[i] == False:
            original_network.remove_node(original_terminal_nodes[i])
    original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]
    print(k)
    print(in_genes_df.count(False))

1
711
2
76
3
17
4
5
5
0


In [302]:

sub_graph = nx.ego_graph(original_network, 'root', radius=n_hidden) #subgraph of neighbors centered at node "root" <= a given radius (n_level).
sub_terminal_nodes = [n for n, d in sub_graph.out_degree() if d == 0]

In [303]:
len(sub_terminal_nodes)

1014

In [304]:
sub_terminal_nodes = [n for n, d in sub_graph.out_degree() if d == 0]
for node in sub_terminal_nodes:
    distance = len(nx.shortest_path(sub_graph, source='root', target=node)) #len of distance: num of nodes in the shortest path
    if (distance == n_hidden + 1) & (node not in original_terminal_nodes):
        part_graph = nx.ego_graph(original_network, node, radius=100)
        corresponding_terminal_nodes = [n for n, d in part_graph.out_degree() if d == 0]
        sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
    elif (distance == n_hidden + 1) & (node in original_terminal_nodes):
        corresponding_terminal_nodes = node
        sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
    elif distance <= n_hidden:
        diff = n_hidden - distance + 1
        sub_graph, copy_node = add_duplicated_edges(sub_graph, node, diff)
        sub_graph = add_gene_edges(sub_graph, copy_node, node, genes_df)
final_network = sub_graph


In [ ]:
train_x.T

In [ ]:
train_y

In [ ]:
diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
train_y['diagnosis_binary'] = train_y['diagnosis'].map(diagnosis_mapping)

# Display the first few rows to verify the mapping
print(train_y.head())
train_y.drop(columns=['diagnosis'], inplace=True)
train_y

In [ ]:
qlabel

In [ ]:
try:
    diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
    qlabel['diagnosis_binary'] = qlabel['diagnosis'].map(diagnosis_mapping)
    
    # Display the first few rows to verify the mapping
    print(qlabel.head())
    qlabel.drop(columns=['diagnosis'], inplace=True)
    qlabel

except:
    print('Previously Run Before')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target

# Example usage



In [ ]:
qlabel

In [ ]:
train_y

In [ ]:
val_x = train_x.T.iloc[40000:,]
val_y = train_y.iloc[40000:,]
train_x = train_x.T.iloc[0:40000,]
train_y = train_y.iloc[0:40000,]


In [ ]:
train_dataset = TabularDataset(train_x,train_y)
val_dataset = TabularDataset(val_x,val_y)
test_dataset = TabularDataset(test_x.T,qlabel)

In [ ]:
val_x

In [ ]:
qlabel

In [ ]:
masking[0].shape, len(layers_node[1])

In [ ]:
masking[1].shape, len(layers_node[2])

In [ ]:
masking[2].shape, len(layers_node[3])

In [ ]:
masking = list(masking.values())
layers_node = list(layers_node.values())

In [ ]:
 import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


#for output_layer in range(2, len(masking) + 2):       -->n
class CustomNetwork(nn.Module):
    def __init__(self, layers_node, n, masking):
        super(CustomNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.masks = []
        self.gamma = 0.0001

        for i in range(n-1):
            #print(-1-i)
            #print(layers_node[-1-i])
            in_features = len(layers_node[-1-i])
            out_features = len(layers_node[-1-i-1])
            self.layers.append(nn.Linear(in_features, out_features, bias=False))
            self.masks.append(masking[-1-i])

        self.layers.append(nn.Linear(len(layers_node[-n]), len(layers_node[0])-1))
        self.masks = [torch.tensor(mask, dtype=torch.float32) for mask in self.masks]


    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            mask = self.masks[i]
            #print(mask[0].shape)
            layer.weight.data *= mask
            #print(layer(x).shape)
            x = layer(x)
            
            '''
            x = x.unsqueeze(2)
            print('x shape before multi: ',x.shape)
            print('shape of mask: ', mask.shape )
            x =  x*mask
            x = x.squeeze(2)
            x = torch.tanh(x)
            '''
            x = torch.relu(x)
            #print('x shape: ',x.shape)
        #print(x.shape)
        
        x = self.layers[-1](x)
        return x


In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    labels_list = []
    
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            #print(outputs)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.append(predicted)
            labels_list.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, predicted_list, labels_list

In [ ]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

def model(train_dataloader , val_dataloader, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50):

    model = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    patience = 20
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    early_stop = False
    
    for epoch in tqdm(range(num_epochs)):
        if early_stop:
            print("Early stopping")
            break
        epoch_cost = 0.
        
        for batch_features,batch_targets in train_dataloader:
            outputs = model(batch_features)
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        
        train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
        val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
        scheduler.step(val_accuracy)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train_accuracy: {train_accuracy}, Val_accuracy: {val_accuracy}')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
        # Save the best model
            torch.save(model.state_dict(), f'best_model_{output_layer}.pth')
            print('Model saved.')
        else:
            epochs_no_improve += 1
    
        # Early stopping
        if epochs_no_improve >= patience:
            early_stop = True
            print("Early stopping triggered")
        
    
    train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
    val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
    
    output_train = (predicted_list_train, labels_list_train)
    output_val = (predicted_list_val, labels_list_val)
    
    return output_train, output_val,model

In [ ]:
torch.round(torch.sigmoid(torch.tensor([0.4,0.8])))

In [ ]:

#from neural_network import *
print_information = True
num_epochs = 500
learning_rate = 0.01
batch_size = 2048
l2_regularization = 0.00001
print_cost = True


# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle= True)
# Example of iterating through the DataLoader


pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))
#masking = list(masking.values())
model_dict = dict()
#layers_node = list(layers_node.values())
activation_output = {}
for output_layer in range(2, len(masking) + 2):
    if print_information:
        print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
    output_train, output_test,model_dict[output_layer] = model(train_dataloader,
                                          val_dataloader,
                                          layers_node,
                                          masking,
                                          output_layer,
                                          learning_rate=learning_rate,num_epochs=num_epochs
                                          )

In [ ]:
model_dict

In [ ]:
model_dict[2]

In [ ]:
model_dict[2].layers[0].weight.shape

In [124]:
masking[0].shape

(28, 135)

In [ ]:
print(model_dict)

In [ ]:
!pip install wandb -Uq

In [ ]:
pred_y_df

In [ ]:
model_dict[2].layers[0].weight

In [84]:
masking[2].shape

(390, 1242)

In [91]:
model_dict[2].layers[0].weight

Parameter containing:
tensor([[0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, 0.0000],
        [-0.0000, 0.0000, -0.0000,  ..., -0.0000, 0.0000, 0.0000],
        [-0.0000, -0.0000, 0.0000,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [-0.0000, 0.0674, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, -0.0000, -0.0000]],
       requires_grad=True)

In [106]:
torch.nonzero(torch.tensor(model_dict[4].layers[2].weight)).squeeze()

/tmp/ipykernel_888033/1061644972.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.nonzero(torch.tensor(model_dict[4].layers[2].weight)).squeeze()


tensor([[  0,  27],
        [  0,  33],
        [  0,  37],
        [  0,  42],
        [  0,  98],
        [  0, 100],
        [  0, 112],
        [  0, 127],
        [  1,   8],
        [  1,  23],
        [  1,  25],
        [  1,  32],
        [  1,  44],
        [  1,  53],
        [  1,  56],
        [  1,  73],
        [  1,  83],
        [  1,  92],
        [  1,  93],
        [  1,  97],
        [  1, 105],
        [  1, 107],
        [  1, 129],
        [  1, 141],
        [  2, 106],
        [  3,   3],
        [  3,  13],
        [  3,  18],
        [  3,  28],
        [  3,  41],
        [  3,  59],
        [  3,  71],
        [  3,  79],
        [  3,  95],
        [  3, 104],
        [  3, 110],
        [  3, 111],
        [  3, 126],
        [  3, 137],
        [  3, 138],
        [  4, 108],
        [  5,  31],
        [  5,  47],
        [  5,  54],
        [  5,  55],
        [  5,  63],
        [  5,  68],
        [  5,  96],
        [  6,   7],
        [  6,  20],


In [107]:
non_zero_indices_flat = torch.nonzero(torch.tensor(masking[0])).squeeze()
non_zero_indices_flat 

tensor([[  0,  27],
        [  0,  33],
        [  0,  37],
        [  0,  42],
        [  0,  98],
        [  0, 100],
        [  0, 112],
        [  0, 127],
        [  1,   8],
        [  1,  23],
        [  1,  25],
        [  1,  32],
        [  1,  44],
        [  1,  53],
        [  1,  56],
        [  1,  73],
        [  1,  83],
        [  1,  92],
        [  1,  93],
        [  1,  97],
        [  1, 105],
        [  1, 107],
        [  1, 129],
        [  1, 141],
        [  2, 106],
        [  3,   3],
        [  3,  13],
        [  3,  18],
        [  3,  28],
        [  3,  41],
        [  3,  59],
        [  3,  71],
        [  3,  79],
        [  3,  95],
        [  3, 104],
        [  3, 110],
        [  3, 111],
        [  3, 126],
        [  3, 137],
        [  3, 138],
        [  4, 108],
        [  5,  31],
        [  5,  47],
        [  5,  54],
        [  5,  55],
        [  5,  63],
        [  5,  68],
        [  5,  96],
        [  6,   7],
        [  6,  20],


In [ ]:
'''
    train_y_df[output_layer] = get_prediction(output_train[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 train_x,
                                                 datatype='diagnosis')

    
    for j in range(len(output_train)):
        ctp_sort = layers_node[0]
        ctp_sort.sort()
        if j != output_layer - 1:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=layers_node[len(layers_node) - 2 - j],
                                                   columns=train_x.columns)
        else:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=ctp_sort,
                                                   columns=train_x.columns)
    activation_output[output_layer] = output_train
    pred_y_df[output_layer] = get_prediction(output_test[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 test_x,
                                                 datatype='diagnosis')
    
pred_test_y = pd.DataFra000me(data=0, index=test_x.columns, columns=['diagnosis'])
pred_test_y['diagnosis'] = pred_y_df.T.describe().T['top']

pred_train_y = pd.DataFrame(data=0, index= train_x.columns, columns=['diagnosis'])
pred_train_y['diagnosis'] = train_y_df.T.describe().T['top']'''

In [ ]:
train_y

In [ ]:
import torch

# Example tensors
x = torch.randn(1024, 216, 1)  # Shape: [1024, 216, 1]
mask = torch.randn(1024, 216, 159)  # Shape: [1024, 216, 159]

# Perform element-wise multiplication
result = mask * x  # Shape: [1024, 216, 159]

# Print shapes
print("x shape:", x.shape)
print("mask shape:", mask.shape)
print("result shape:", result.shape)



In [ ]:
labels = torch.randint(0, 2, (100,))  
labels

In [ ]:
mask.shape

In [ ]:
result

In [108]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[ 1.8656, -0.9858]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[ 0.9532, -0.7556]], grad_fn=<TanhBackward0>)


In [109]:
layer_output.shape

torch.Size([1, 2])

In [ ]:
sum(pred_train_y.diagnosis.values == rlabel.diagnosis.values)/len(pred_train_y)

In [ ]:
sum(pred_test_y.diagnosis.values == qlabel.diagnosis.values)/len(pred_test_y )

In [ ]:
qlabel

In [110]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[0.0911, 0.1891]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[0.0908, 0.1869]], grad_fn=<TanhBackward0>)


In [111]:
layer_output

tensor([[0.0911, 0.1891]], grad_fn=<AddmmBackward0>)

In [117]:
import torch
import torch.nn as nn

# Step 1: Define the Linear Layer
layer = nn.Linear(3, 2)

# Initialize layer weights for demonstration
nn.init.constant_(layer.weight, 5.0)
nn.init.constant_(layer.bias, 0.0)

print("Original weights:")
print(layer.weight)

# Step 2: Create a Mask Matrix
# Example mask matrix: 2x3, matching the shape of layer.weight
mask = torch.tensor([[1, 0, 1],
                     [0, 1, 0]], dtype=torch.float32)

print("Mask matrix:")
print(mask)

# Step 3: Apply the Mask
# Element-wise multiplication of the layer's weights and the mask
layer.weight.data *= mask

print("Sparse weights:")
print(layer.weight)

# Now you can use the layer in your model as usual.


Original weights:
Parameter containing:
tensor([[5., 5., 5.],
        [5., 5., 5.]], requires_grad=True)
Mask matrix:
tensor([[1., 0., 1.],
        [0., 1., 0.]])
Sparse weights:
Parameter containing:
tensor([[5., 0., 5.],
        [0., 5., 0.]], requires_grad=True)


In [120]:
masking[0].shape

(28, 135)

In [112]:
layer.weight

Parameter containing:
tensor([[-0.5483,  0.3901,  0.0170],
        [ 0.1705,  0.3766, -0.3806]], requires_grad=True)

In [113]:
layer.weight.shape

torch.Size([2, 3])

In [114]:
x.shape

torch.Size([1, 3])

In [115]:
-0.1207*1 + 0.5761*2 -0.0801*3

0.7911999999999999

In [ ]:
-0.1207,  0.5761, -0.0801

In [ ]:
(-0.3824*1) +  (0.5135*2) -(0.1200*3)

In [ ]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2, bias = False)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

In [ ]:
layer.weight.shape

In [ ]:
x.shape

In [ ]:
# What nn.linear does

torch.matmul(x,layer.weight.T)    


In [ ]:
layer.weight

In [ ]:
1*0.5236 -0.5384*2-0.4127*3

In [ ]:
layer_output

In [ ]:
import torch

# Create a batch of tensors
num_samples = 1024
num_features = 3
x_batch = torch.randn(num_samples, num_features)  # Shape: [1024, 3]§

# Print the shape and content of the batch
print("x_batch shape:", x_batch.shape)
print("x_batch tensor:", x_batch)


In [ ]:
layer = nn.Linear(3, 2, bias = False) 

In [ ]:
layer.weight

In [ ]:
layer_output = layer(x_batch)

In [ ]:
layer_output 

In [ ]:
1.3851 * -0.1641 + 0.2640 * 0.5494 + 0.1831 * -1.0035

# Understading Masking and batch intorch
